<a href="https://colab.research.google.com/github/melmar-g1thub/INTERPOLATING-NEURAL-NETWORK/blob/main/local_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of a localized training approach for neural network interpolation.

Specifically, it trains the model on small triplets of neighboring data points, allowing the learning of the network on a local scale. By focusing on triplets, the function isolates the network's ability to learn smooth mappings between closely related inputs and outputs, avoiding global patterns and emphasizing local generalization.

### PREPARE TRIPLETS AND TRAIN

In [ ]:
gap = 1
# Training triplets
in_train_triplets = torch.stack([
    torch.cat((in_train_tensor[i - gap], in_train_tensor[i + gap]))
    for i in range(gap, len(in_train_tensor) - gap)
])
out_train_targets = out_train_tensor[gap : len(out_train_tensor) - gap]

# Validation triplets
in_val_triplets = torch.stack([
    torch.cat((in_val_tensor[i - gap], in_val_tensor[i + gap]))
    for i in range(gap, len(in_val_tensor) - gap)
])
out_val_targets = out_val_tensor[gap : len(out_val_tensor) - gap]

In [ ]:
# Define some necessary functions for triplet interpolation
# Function for performing inverse transform to predictions to get physical values
def inv_log_scaled(values, scaler):
    real_val = scaler.inverse_transform(np.array(values).reshape(-1, 1)).flatten()
    physical_val = 10**real_val
    return physical_val

# Get triplet indices
def get_interpolation_indices(center_index, data_length, gap):
    idx1 = center_index - gap
    idx3 = center_index + gap
    if idx1 < 0 or idx3 >= data_length:
        return None
    return idx1, center_index, idx3

# Function to load activation from string
def load_activation_fn(activation_str):
    activation_map = {
        "ReLU": nn.ReLU,
        "LeakyReLU": nn.LeakyReLU,
        "SiLU": nn.SiLU
    }

    if "LeakyReLU" in activation_str:
        return activation_map["LeakyReLU"]
    elif "ReLU" in activation_str:
        return activation_map["ReLU"]
    elif "SiLU" in activation_str:
        return activation_map["SiLU"]
    else:
        raise ValueError(f"Activation function '{activation_str}' not recognized.")

In [ ]:
top_path = "/content/drive/My Drive/Colab Notebooks/Random_16_06"
output_dir  = os.path.join(top_path, "Triplet")
os.makedirs(save_dir, exist_ok=True)

config_path = "/content/drive/My Drive/Colab Notebooks/Random_16_06/top1/config.json"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open(config_path, "r") as f:
    top1_config = json.load(f)

hidden_sizes  = top1_config.get("hidden_layer_sizes")
dropout  = top1_config.get("dropout")
activation_str  = top1_config.get("activation_fn")
activation_fn  = load_activation_fn(activation_str)
lr = top1_config.get("learning_rate")

input_triplet = 4 # concate x1, x3 = [T1, nb1, T3, nb3]
output_triplet = 2 # P, S

# Instantiate the model, without  previous weights just hiperparameters !!!
triplet_model = Interpolation(in_size = input_triplet,
                              out_size = output_triplet,
                              hidden_layer_sizes=hidden_sizes,
                              dropout=dropout,
                              activation=activation_fn).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(triplet_model.parameters(), lr)

In [ ]:
def train_triplet_model(model, in_train, out_train, in_val, out_val,
                        criterion, optimizer, device, epochs, save_path):
    best_val_loss = float("inf")
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        preds_train = model(in_train.to(device))
        loss_train = criterion(preds_train, out_train.to(device))

        loss_train.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            preds_val = model(in_val.to(device))
            loss_val = criterion(preds_val, out_val.to(device))

        train_losses.append(loss_train.item())
        val_losses.append(loss_val.item())

        print(f"Epoch {epoch+1:03d} | Train Loss: {loss_train.item():.5e} | Val Loss: {loss_val.item():.5e}")

        if loss_val.item() < best_val_loss:
            best_val_loss = loss_val.item()
            torch.save(model.state_dict(), save_path)
            print(f"  Best model saved (val_loss = {best_val_loss:.5e})")

    return train_losses, val_losses

In [ ]:
save_path  = os.path.join(output_dir, "best_triplet_model.pth")

train_losses, val_losses = train_triplet_model(
    model=triplet_model,
    in_train=in_train_triplets,
    out_train=out_train_targets,
    in_val=in_val_triplets,
    out_val=out_val_targets,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=150,
    save_path=save_path
)

losses_df = pd.DataFrame({
    "epoch": list(range(1, len(train_losses) + 1)),
    "train_loss": train_losses,
    "val_loss": val_losses
})

losses_csv_path = os.path.join(output_dir, "triplet_model_losses.csv")
losses_df.to_csv(losses_csv_path, index=False)


plt.figure(figsize=(10, 6))
plt.plot(losses_df["epoch"], losses_df["train_loss"], label="Train Loss", linewidth=2)
plt.plot(losses_df["epoch"], losses_df["val_loss"], label="Validation Loss", linewidth=2)
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("TripletNet Training and Validation Loss")
plt.yscale("log")
plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "triplet_model_loss_curve.png"), dpi=300)
plt.show()

### NN and B-SPlines INTERPOLATION

In [ ]:
# #######################################################
# INTERPOLATION FOR THE TRIPLET
# #######################################################
def Interpolation_Triplet_test(model, inputs_set, outputs_set, scaler_P, scaler_S, device, indices, save_dir, gap):
    model.eval()
    os.makedirs(save_dir, exist_ok=True)
    metrics = []

    model = model.to(device)

    for i, center_index in enumerate(indices):
        start_time = time.time()
        idx1, idx2, idx3 = get_interpolation_indices(center_index, len(inputs_set), gap)

        if idx1 is None:
            print(f"Saltando índice {center_index} (gap={gap}) debido a límites.")
            continue

        x1 = inputs_set[idx1]
        x3 = inputs_set[idx3]

        y1 = outputs_set[idx1]
        y2_true = outputs_set[idx2]
        y3 = outputs_set[idx3]

        # Concatenate x1 y x3 for input in the Interpolation NN
        triplet_input_tensor = torch.cat([
            torch.FloatTensor(x1).unsqueeze(0),
            torch.FloatTensor(x3).unsqueeze(0)
        ], dim=1).to(device)

        # Real outputs
        P_true = 10 ** np.array([y1[0], y2_true[0], y3[0]])
        S_true = 10 ** np.array([y1[1], y2_true[1], y3[1]])

        # Predictions
        with torch.no_grad():
            y2_pred_scaled = model(triplet_input_tensor).cpu().numpy()[0]

        P2_pred = inv_log_scaled(y2_pred_scaled[0], scaler_P)
        S2_pred = inv_log_scaled(y2_pred_scaled[1], scaler_S)

        # Metrics
        mse_P = mean_squared_error([P_true[1]], [P2_pred])
        mae_P = mean_absolute_error([P_true[1]], [P2_pred])
        mse_S = mean_squared_error([S_true[1]], [S2_pred])
        mae_S = mean_absolute_error([S_true[1]], [S2_pred])

        metrics.append({
            "method": "TripletNet", "index": int(center_index),
            "gap": gap,
            "mse_P": mse_P, "mae_P": mae_P,
            "mse_S": mse_S, "mae_S": mae_S
        })

        elapsed = time.time() - start_time
        for m in metrics:
            m["elapsed_sec"] = elapsed


        # Interpolation graph
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        x_labels = [f'Idx {idx1}', f'Idx {idx2}', f'Idx {idx3}']
        x_ticks = np.arange(3)

        axs[0].plot(x_ticks, P_true, label='True P', marker='o', color='green')
        axs[0].scatter(x_ticks[1], P2_pred, label='Predicted P', marker='x', s=100, color='red')
        axs[0].set_title('Pressure (P) [MeV/fm³]', fontsize=14)
        axs[0].set_xticks(x_ticks)
        axs[0].set_xticklabels(x_labels, fontsize=14)
        axs[0].tick_params(axis='y', labelsize=14)
        axs[0].grid(True)
        axs[0].legend(fontsize=14)
        axs[0].ticklabel_format(axis='y', style='sci', scilimits=(-3, 3))

        axs[1].plot(x_ticks, S_true, label='True S', marker='o', color='blue')
        axs[1].scatter(x_ticks[1], S2_pred, label='Predicted S', marker='x', s=100, color='orange')
        axs[1].set_title('Entropy (S) [1/fm³]', fontsize=14)
        axs[1].set_xticks(x_ticks)
        axs[1].tick_params(axis='y', labelsize=14)
        axs[1].set_xticklabels(x_labels, fontsize=14)
        axs[1].grid(True)
        axs[1].legend(fontsize=14)
        axs[1].ticklabel_format(axis='y', style='sci', scilimits=(-3, 3))

        plt.suptitle(f"NN Local Interpolation Test #{i+1} (Index {center_index})  (Gap {gap})", fontsize=14)
        plt.tight_layout()
        filename = f"TripletNet_triplet_{i+1}_index_{center_index}.png"
        plt.savefig(os.path.join(save_dir, filename), dpi=300)
        plt.close()

    return metrics

In [ ]:
#######################################################
# b-SPLINES INTERPOLATION
#######################################################
def interpolation_test_bspline(in_train, out_train, in_test, out_test, indices, save_dir, gap):
    os.makedirs(save_dir, exist_ok=True)

    x = in_train[:, 0]
    y = in_train[:, 1]
    z_P = out_train[:, 0]
    z_S = out_train[:, 1]

    bspline_P = SmoothBivariateSpline(x, y, z_P, kx=3, ky=3)
    bspline_S = SmoothBivariateSpline(x, y, z_S, kx=3, ky=3)

    metrics = []

    for i, center_index in enumerate(indices):
        if center_index < 1 or center_index >= len(in_test) - 1:
            continue

        start_time = time.time()

        x1 = in_test[center_index - gap]
        x2 = in_test[center_index]
        x3 = in_test[center_index + gap]

        y1 = out_test[center_index - gap]
        y2_true = out_test[center_index]
        y3 = out_test[center_index + gap]

        # Predicions for middle point
        P_pred_log = bspline_P.ev(x2[0], x2[1])
        S_pred_log = bspline_S.ev(x2[0], x2[1])

        # Inverse log transformation to get real phyisical values
        P_true = 10 ** np.array([y1[0], y2_true[0], y3[0]])
        S_true = 10 ** np.array([y1[1], y2_true[1], y3[1]])
        P_pred = 10 ** P_pred_log
        S_pred = 10 ** S_pred_log

        # Metrics
        mse_P = mean_squared_error([P_true[1]], [P_pred])
        mae_P = mean_absolute_error([P_true[1]], [P_pred])
        mse_S = mean_squared_error([S_true[1]], [S_pred])
        mae_S = mean_absolute_error([S_true[1]], [S_pred])

        metrics.append({
            "method": "B-Spline",
            "index": int(center_index),
            "mse_P": mse_P,
            "mae_P": mae_P,
            "mse_S": mse_S,
            "mae_S": mae_S
        })

        elapsed = time.time() - start_time
        for m in metrics:
            m["elapsed_sec"] = elapsed

        # Plot
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        x_labels = [f'Idx {center_index-gap}', f'Idx {center_index}', f'Idx {center_index+gap}']
        x_ticks = np.arange(3)

        axs[0].plot(x_ticks, P_true, label='True P', marker='o', color='green')
        axs[0].scatter(x_ticks[1], P_pred, label='Predicted P', marker='x', s=100, color='red')
        axs[0].set_title('Pressure (P) [MeV/fm³]', fontsize=14)
        axs[0].set_xticks(x_ticks)
        axs[0].set_xticklabels(x_labels, fontsize=14)
        axs[0].tick_params(axis='y', labelsize=14)
        axs[0].grid(True)
        axs[0].legend(fontsize=14)
        axs[0].ticklabel_format(axis='y', style='sci', scilimits=(-3, 3))

        axs[1].plot(x_ticks, S_true, label='True S', marker='o', color='blue')
        axs[1].scatter(x_ticks[1], S_pred, label='Predicted S', marker='x', s=100, color='orange')
        axs[1].set_title('Entropy (S) [1/fm³]', fontsize=14)
        axs[1].set_xticks(x_ticks)
        axs[1].tick_params(axis='y', labelsize=14)
        axs[1].set_xticklabels(x_labels, fontsize=14)
        axs[1].grid(True)
        axs[1].legend(fontsize=14)
        axs[1].ticklabel_format(axis='y', style='sci', scilimits=(-3, 3))

        plt.suptitle(f"B-Spline Local Triplet Test #{i+1} (Index {center_index})", fontsize=14)
        plt.tight_layout()
        filename = f"SPLINE_triplet_{i+1}_index_{center_index}.png"
        plt.savefig(os.path.join(save_dir, filename), dpi=300)
        plt.close()

    return metrics

In [ ]:
# Upload top1 model hyperparameteres
output_dir  = "/content/drive/My Drive/Colab Notebooks/Random_16_06/Triplet"
best_model_path = os.path.join(output_dir , "best_triplet_model.pth")

triplet_model.load_state_dict(torch.load(best_model_path))
triplet_model.eval()

# Define indices
print(f"Indices for InterpolationTripletNe from test data set: {indices}")

# Define saving directory
triplet_nn_save_base_dir = output_dir
os.makedirs(triplet_nn_save_base_dir, exist_ok=True)

In [ ]:
for current_gap in [1, 5]:
    print(f"\n▶️ TripletNet and B-Spline with gap={current_gap}...")

    save_dir_gap = os.path.join(triplet_nn_save_base_dir, f"gap_{current_gap}")
    os.makedirs(save_dir_gap, exist_ok=True)

    #### TripletNet ####
    metrics_triplet_nn_gap = Interpolation_Triplet_test(
        model=triplet_model,
        inputs_set=in_test,
        outputs_set=out_test,
        scaler_P=scaler_P,
        scaler_S=scaler_S,
        device=device,
        indices=indices,
        save_dir=save_dir_gap,
        gap=current_gap
    )
    df_nn = pd.DataFrame(metrics_triplet_nn_gap)

    #### B-Spline ####
    metrics_spline_gap = interpolation_test_bspline(
        in_train=in_train,
        out_train=out_train,
        in_test=in_test,
        out_test=out_test,
        indices=indices,
        save_dir=save_dir_gap,
        gap=current_gap
    )

    df_spline = pd.DataFrame(metrics_spline_gap)

    df_all = pd.concat([df_nn, df_spline], ignore_index=True)
    csv_path = os.path.join(save_dir_gap, f"metrics_triplet_gap_{current_gap}.csv")
    df_all.to_csv(csv_path, index=False)